### 1. 正则的几个用法

1. 匹配手机号

In [6]:
import re

phone_p = re.compile('\d{3}[-\s]?\d{4}[-\s]?\d{4}')
result = re.findall(phone_p, 'Myphone is :135-0009-4747 or 188 4786 8856 or 15878963367')
print(result)

['135-0009-4747', '188 4786 8856', '15878963367']


2.贪婪匹配和非贪婪匹配

- 贪婪匹配: *

- 非贪婪匹配(可能为0个)： *?

- 贪婪匹配所有字符:  .*  

- 非贪婪匹配所有字符: .*? 

In [7]:
import re
p1 = re.compile(r'我爱好.*')
result = re.match(p1, '我爱好好好好')
print(result)

p2 = re.compile(r'我爱好.*?')
result = re.match(p2, '我爱好好好好')
print(result)

<_sre.SRE_Match object; span=(0, 6), match='我爱好好好好'>
<_sre.SRE_Match object; span=(0, 3), match='我爱好'>


3.使用捕获组

括号的使用
findall可以用于找只有一个匹配内容的，相当于只有一个括号的情况，但是当有多个括号的时候，使用findall就会比较麻烦

In [8]:
p = re.compile(r'[a-z]+((\d+)-(\d+))[a-z]+')
 
result = re.match(p, 'afs123-456gds')
print(result.group()) # afs123-456gds
print(result.group(1)) # 123-456
print(result.group(2)) # 123
print(result.group(3)) # 456

afs123-456gds
123-456
123
456


4.字符串的替换

In [9]:
import re
 
p = re.compile(r'\d+')
 
result = re.sub(p, '中文', 'tom32jerry456haha')
print(result) # tom中文jerry中文haha

tom中文jerry中文haha


### 2.构建谷歌搜索函数

- 关键点1:如果实现翻墙？在本地使用了小飞机，如何让我们的python请求也经过小飞机？答案是使用代理，小飞机的代理端口是1080
所以只需要将代理指定到本地的1080即可

- 关键点2:仔细观察谷歌搜索时向服务器发送的请求，可以获得最近一年，最近一个月最近一周的结果


In [2]:
import re
import requests 
from bs4 import BeautifulSoup
proxy = {"http":"http://127.0.0.1:1080","https":"https://127.0.0.1:1080"}
headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }
phone_p = re.compile('\d{3}[-\s]?\d{4}[-\s]?\d{4}')

def google(q):
    s = requests.Session()
    q = '+'.join(q.split())
    tbs = '&tbs=qdr:y'
    url = 'https://www.google.com/search?q=' + q + '&ie=utf-8&oe=utf-8'
#     url = 'https://www.google.com/search?q=' + q + '&start=10&ie=utf-8&oe=utf-8'
    r = s.get(url, headers=headers_Get,proxies = proxy,verify=False)

    soup = BeautifulSoup(r.text, "html.parser")
    print(soup.find('div', id='resultStats'))
    output = []
#     for searchWrapper in soup.find_all('h3', {'class':'r'}): #this line may change in future based on google's web page structure
#         url = searchWrapper.find('a')["href"] 
#         text = searchWrapper.find('a').text.strip()
#         result = {'text': text, 'url': url}
#         output.append(result)
    
    for content_p in soup.find_all('span', {'class':'st'}):
        text = content_p.text
        result = {'text': text}
        output.append(result)
        print(text)
        resultm = re.findall(phone_p, text)
        print(resultm)
        print("\n")
    return output

(google("hello"))

C:\Users\mk10730\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<div id="resultStats">About 1,250,000,000 results<nobr> (0.46 seconds) </nobr></div>
Connect with people who share your interests. Discover communities you love. Join fun conversations.
[]


3 days ago - HELLO! US brings you the latest celebrity & royal news from around the world, magazine exclusives, celeb babies, weddings, pregnancies and ...
[]


"Hello" is a song by English singer Adele, released on 23 October 2015 by XL Recordings as the lead single from her third studio album, 25 (2015). Adele ...
[]


Hello definition, (used to express a greeting, answer a telephone, or attract attention.) See more.
[]


Shop our naturally friendly products for your family here. Our vegan friendly toothpaste is free from dyes, artificial sweeteners and other preservatives.
[]


The greeting hello is among the most generic and neutral in use. It may be heard in nearly all social situations and in nearly all walks of life, and is unlikely to ...
[]


Hello is a creative agency driven to craft wort

[{'text': 'Connect with people who share your interests. Discover communities you love. Join fun conversations.'},
 {'text': '3 days ago - HELLO! US brings you the latest celebrity & royal news from around the world, magazine exclusives, celeb babies, weddings, pregnancies and\xa0...'},
 {'text': '"Hello" is a song by English singer Adele, released on 23 October 2015 by XL Recordings as the lead single from her third studio album, 25 (2015). Adele\xa0...'},
 {'text': 'Hello definition, (used to express a greeting, answer a telephone, or attract attention.) See more.'},
 {'text': 'Shop our naturally friendly products for your family here. Our vegan friendly toothpaste is free from dyes, artificial sweeteners and other preservatives.'},
 {'text': 'The greeting hello is among the most generic and neutral in use. It may be heard in nearly all social situations and in nearly all walks of life, and is unlikely to\xa0...'},
 {'text': 'Hello is a creative agency driven to craft worthy experien

In [3]:
#coding:utf-8
import requests
import json
import time
import re

headers = {
    'Host':"map.baidu.com",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.5",
    "Connection": "keep-alive",
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:39.0) Gecko/20100101 Firefox/39.0"}

def citys():
    html=requests.get('http://map.baidu.com/?newmap=1&reqflag=pcmap&biz=1&from=webmap&da_par=baidu&pcevaname=pc4.1&qt=s&da_src=searchBox.button&wd=%E6%B1%BD%E8%BD%A6%E7%BE%8E%E5%AE%B9%E5%BA%97&c=1&src=0&wd2=&sug=0&l=5&b=(7002451.220000001,1994587.88;19470675.22,7343963.88)&from=webmap&biz_forward={%22scaler%22:1,%22styles%22:%22pl%22}&sug_forward=&tn=B_NORMAL_MAP&nn=0&u_loc=12736591.152491,3547888.166124&ie=utf-8&t=1459951988807',headers=headers).text
    f=open('city_ids.txt','a')
    data=json.loads(html)
    for item in data['content']:
        #for city in item['city']:
            f.write(str(item)+'\n')
    f.close()

def get_infor(keyword,code,page):
    html=requests.get('http://map.baidu.com/?newmap=1&reqflag=pcmap&biz=1&from=webmap&da_par=baidu&pcevaname=pc4.1&qt=con&from=webmap&c='+str(code)+'&wd='+keyword+'&wd2=&pn='+str(page)+'&nn='+str(page*10)+'&db=0&sug=0&addr=0&&da_src=pcmappg.poi.page&on_gel=1&src=7&gr=3&l=12&tn=B_NORMAL_MAP&u_loc=12736591.152491,3547888.166124&ie=utf-8',headers=headers).text
    data=json.loads(html)['content']
    return data


def main():
    keys=['深圳 横岗 广告']
    for keyword in keys:
        f=open(keyword+'_tels.txt','a')
        code=340
        page=1
        while True:
            try:
                data=get_infor(keyword,code,page)
            except:
                break
            if data==[]:
                break
            for item in data:
                f.write(str(item['area_name'])+',    ')
                f.write(str(item['name'])+',    ')
                f.write(str(item['addr'])+',    ')
                f.write(str(item['ext']['detail_info']['phone'])+'\n')
                # print(str(item['area_name'])+'\n')

            page+=1
            print(code,page)
            time.sleep(1)
        f.close()
main()

340 2
340 3
340 4
340 5
340 6
340 7
340 8
340 9


In [4]:
def getWeChat():
    keys=['深圳 横岗 广告']
    for keyword in keys:
        code=340
        page=1
        f=open(keyword+'_tels.txt','a')
        while True:
            try:
                data=get_infor(keyword,code,page)
            except:
                break
            if data==[]:
                break
            for item in data:
                print(str(item['name'])+' ')
                google(str(item['area_name'])+' '+ str(item['name'])+' 联系方式')

            page+=1
            print(code,page)
            time.sleep(1)         
        f.close()
getWeChat()

纪元星广告装饰有限公司 


C:\Users\mk10730\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<div id="resultStats">About 20,200 results<nobr> (0.43 seconds) </nobr></div>
这里是深圳市简逸风装饰设计工程有限公司企业简介,找到网为您免费提供深圳市简逸风装饰设计工程有限公司 ... 企业地址：, 深圳市龙岗区龙岗街道龙园5巷21号530.
[]


这里是深圳市子墨堂广告有限公司企业简介,找到网为您免费提供深圳市子墨堂广告有限公司地址电话,员工 ... 联系邮箱. 邮箱: 26114988@qq.com. 年报邮箱. 地址: 深圳市龙岗区龙岗街道盛平社区龙西东路50号 ... 湛江市艺之峰装饰有限公司 ... 子墨堂广告有限公司，位于深圳市龙岗区龙岗中心城区，也是2011年大运会举办的城市。
[]


Feb 15, 2012 - 深圳市大虹星装饰工程有限公司遗失国、地税税务登记证副本，识别 ... 楼歌舞厅场地2520m2西侧800m2LED广告场地招租电话13923777161江 ... 执业地址：福田区深南中路6011号绿景纪元大厦A座16层，注册登记 ... 联系电话13612818665联系人：王玮，地址：深圳市龙岗区中心城宝钜大厦1楼B101－1号.
['13923777161', '13612818665']


深圳市龙岗区龙岗广告公司，公司主要业务为户内外广告、印刷喷绘、画册彩页设计 ... 工艺、公司工厂挂画；商场包装策划；灯箱广告；公司前台背景装饰广告牌、公司 ...
[]


Jul 23, 2016 - 据介绍，7 月21 日15 时38 分，龙岗区坂田信义嘉御山，深圳市纪元星广告装饰有限公司进行设立广告牌时，施工人员使用大型机械开挖造成地下中 ...
[]


深圳市龙岗区胜友光电厂的诚信、实力和产品质量获得业界的认可。欢迎各界朋友 ... 30CM 50CM 80CM led流星雨灯管led七彩挂树流星灯串彩灯装饰批发 · 深圳市龙岗区胜 ... 市罗湖区. 户外草坪防水插卡Logo广告投影灯激光灯led酒吧舞台灯光设备 ... 10米100灯led彩灯串灯户外防水公母接led圣诞节日满天星装饰灯串 · 深圳市龙岗 ...
[]


Jul 23, 2016 - 据介绍，7月21日15时38分，龙岗区坂田信义嘉御山，深圳市纪元星广告装饰有限公司进行设立广告

C:\Users\mk10730\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<div id="resultStats">About 19,600,000 results<nobr> (0.53 seconds) </nobr></div>
深圳胜希林广告喷绘有限公司. 联系人：张总手机：13728888532 电话：0755-82448600 网址：530742544@qq.com 地址：深圳市龙岗区布吉街道甘坑社区甘碧路华光 ...
['13728888532', '755-82448600']


深圳市龙岗区鸿鑫广告部位于深圳市布吉街道吉政路。本公司是一家专业从事标牌制作、灯箱广告、广告标识、广告喷绘等户内外广告喷绘制作公司，拥有先进的设备。
[]


深圳市鸿鑫广告有限公司成立于2004年，是一家专业从事广告设计,活动策划，现场 ... 喷绘写真. 联系地址：深圳市龙岗区布吉吉政路中腾盛科技园A栋303 联系方式：
[]


May 24, 2016 - 联系方式. 深圳市鸿盛广告有限公司 电话：0755-8482 1239 ... 深圳市鸿盛广告有限公司（鸿盛广告）成立于2008年，在深圳龙岗广告公司中，龙岗鸿盛广告在招牌制作，LED发光字制作，喷绘、设计等有着卓越 ... 联系电话：13360071239 温小姐或在线应聘 ... 公司地址：深圳市龙岗区深惠公路（南联段-上龙塘路口63号）.
['755-8482 1239', '13360071239']


... 厂家和供应商。采购万能打印机，UV平板打印机，万能平板打印机相关产品请联系深圳恒诚伟业科技有限公司。 ... 手机壳uv平板打印机3d彩印喷绘机深圳厂家直销.
[]


地址： 深圳市龙岗区坂田街道吉华路上雪科技园北区1号J栋 ..... 主营产品： 深圳龙岗广告喷绘，深圳广告喷绘公司，深圳日晟广告喷绘，龙岗布吉广告喷绘，深圳福田 ...
[]


深圳市宏盛龙润彩印机械设备有限公司(彩盛UV打印机)是一家集UV打印机、UV平板机研发设计、 ... 深圳市龙岗区平湖街道新木社区乐新路54号 ... 老品牌（彩神/FLORA）全系列uv打印机,UV平板打印机,UV平板喷绘机,广告喷绘机,瓷砖彩雕背景打印机, ... 联系我们. 联系人：彩盛UV打印机. 手机：0755-82226789. 电话：0755-81116789.
['755-82226789', '755

...

<div id="resultStats">About 8 results<nobr> (0.55 seconds) </nobr></div>
BOSS直聘为求职者提供2018年最新欣乐美招聘，更有欣乐美的公司简介、公司地址、产品介绍、公司环境、CEO高管 ... 来源：企查查深圳市欣乐美文化传播有限公司.
[]


全媒通为全国各地广告主提供深圳市户外大牌广告投放、深圳户外墙体广告价格、深圳户外大牌广告报价 ... 媒体位置：深圳市罗湖区深南大道与人民路交汇处媒体形式：玻璃幕墙单透媒体 .... 4、媒体可进行特殊异形效果发布，可全方位150°环视华强路与深南中路; .... 媒体位置, 深圳市龙岗区中心城天安数码城对面清林路与黄阁路交汇处.
[]


龙岗广告公司联系方式：0755-28942291. ... 联系我们. 地址：深圳市龙岗区盛平社区龙西东路50号（盛平社区工作站旁）. 电话：0755-28942291 传真：0755-28934189
['755-28942291', '755-28942291', '755-28934189']


中国深圳人力资源服务智慧广场公共服务大厅是集人力资源、社会保障、公安户政、 .... 资源服务智慧广场管委办主办，北京外企德科人力资源服务深圳有限公司承办的 ... 智慧广场公益大讲堂致力通过全方位的人力资源管理类培训，帮助学员们从每一期 ... 年5月28日（星期一）14:00—17:00四、活动地点深圳市龙岗区天安云谷产业园一 ...
[]


龙岗区党群服务中心和新时代讲习所启用2018-07-05. 龙岗区举行扫黑除恶专项斗争宣传工作推进会2018-07-05. 中国国际多肽会议首次在龙岗召开2018-07-05.
[]


龙岗区人力资源局“三个精心”积极开展劳动法律宣传月系列活动2018-06-07; 我局赴 ... 我局李相宏局长率队走访调研深圳秋田微电子股份有限公司2018-04-27; 我局 ...
[]


广州市艾依格家居制品有限公司; 公司地址（总部）：广州市白云区太和镇南岭工业区中路12号（地铁3号线龙归站附近）; 邮编：510445; 加盟咨询：4000 222 660 (免费 ...
[]


深圳快三秒计算机有限公司是香港快三秒电脑集团旗下的一家全资子公司。 ... 直属工厂：香港

C:\Users\mk10730\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<div id="resultStats">About 853,000 results<nobr> (0.47 seconds) </nobr></div>
QQ邮 箱：2237747589@qq.com. 公司网址：www.szxtbxg.com. 公司地址：深圳市龙岗区横岗街道红棉四路36号一楼. 东莞市新泰不锈钢制品有限公司. 联系人：余先生.
[]


联系电话：0755-27276628. 网 址：www.szxtbxg.com. 地 址：深圳市龙岗区横岗街道红棉四路36号一楼. 组合式不锈钢水箱,深圳市新泰不锈钢制品有限公司 当前位置： ...
['755-27276628']


深圳市帅泰科技有限公司 服务热线：0755-84586001、8458602、84586003. 公司地址：深圳市龙岗区龙岗街道宝龙社区宝龙工业城诚信路8号亚森工业厂区4号厂房1 ...
['755-84586001']


深圳市捷丰泰科技有限公司专业家研发、生产、销售各种型号的数控广告雕刻机、木工雕刻机、大理石 ... 捷丰泰科技自成立以来，以良好而广泛的社会信誉为保障，在日常运营中努力树立高品质的企业形象， ... 深圳市龙岗区 ..... 阿里巴巴为您找到深圳市捷丰泰科技有限公司推荐的产品,包括图片，价格，交易以及在线联系方式等信息。
[]


深圳市睿泰广告有限公司，平湖街道凤凰社区鸿盛街18号，主要经营广告标识;激光 ... 广告招牌;广告发光字;无边字迷你字，请联系我们深圳市睿泰广告有限公司，一流 ... 深圳市睿泰广告有限公司、座落于国内最大最知名的广告标识基地、位置深圳市龙岗区、 .... 联系方式. 周武锋 先生 （营销部经理） 旺旺在线. 电 话：: 86 0755 28499669.
['755 28499669']


指尖网络工作室成立于2016年，是专门为企业网站建设及微信公众号广告推广，联系人是 ... 联系人：: 罗如华; 区号：: 查询区号; 电话：: 17688984984; 邮箱： ... 山东新泰圣雅纺织有限公司; · 济南凯昌伟业环保科技有限公; · 汕头市澄海区思杰玩具商行 ... 深圳市广瑞新材料有限公司; 吉林民丰米业有限公司; 深圳市小车便利汽车服务有限 ...
['17688984984']


这里是深圳市地天泰礼业科技

C:\Users\mk10730\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


<div id="resultStats">About 5 results<nobr> (0.69 seconds) </nobr></div>
地址：深圳市宝安区宝田一路12号公交站：宝安西乡流塘派出所、宝安中心客运站、宝安西乡城管办等都可到达地铁站：环中线到灵芝站、洪浪北站、兴东站出站后转乘 ...
[]


联系方式. 公司名称: 深圳市率先电器有限公司. 公司地址: 深圳市龙岗区横岗镇大康龙兴路三巷10号. 邮政编码: 518115. 电话: 0755-28657516. 传真: 86-0755- ...
['755-28657516']


联系电话：0755-83860721. 店铺地址：深圳市福田区福田街道 ... 店铺地址：深圳市宝安区宝安龙华镇人民北路291号佳华商场一楼KISSCAT专柜 ... 店铺地址：深圳市宝安区新安街道办新安5区海雅缤纷城海雅（商铺部分）2层KISSCAT专柜 ... 店铺地址：深圳市龙岗区横岗街道办湛宝路信义湛宝大厦龙洲百货旁横岗天虹1层KISSCAT专柜 ...
['755-83860721']


深圳登喜路酒店（宝安店）是法国登喜路酒店集团在大中华区倾力打造的星级标准酒； ... 广东省深圳市南山区区；深圳登喜路酒店(南山店)电话；深圳登喜路酒店(南山店) ... 在大中华区倾力打造的星级标准酒店，深圳登喜路酒店(南山店)位于深圳市宝安区新安 ... 设施齐全，装修浪漫高雅，融欧式古典建筑风格和现代设计为一体，别俱一格。
[]


横岗附近好吃的美食有哪些,大众点评为您找到3167家好吃的横岗美食餐厅,找深圳 ... 火锅 | 横岗 龙岗大道2999号麟恒中心广场二期二层 ... 优品阁概念餐厅(横岗店).
[]


340 9
